In [61]:
import tensorflow as tf
print(tf.__version__)


2.12.0


In [62]:
import numpy as np
import pandas as pd
import tensorflow as tf
# from tensorflow.python.keras import models
from keras import regularizers
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dense, Dropout, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.cluster import KMeans

In [63]:
# 讀取訓練和測試資料
train_label_csv = pd.read_csv("Arrhythmia Data Set-20230503T055529Z-001/Arrhythmia Data Set/train_label.csv", header=None)
train_data_csv = pd.read_csv("Arrhythmia Data Set-20230503T055529Z-001/Arrhythmia Data Set/train_data.csv", header=None)
test_label_csv = pd.read_csv("Arrhythmia Data Set-20230503T055529Z-001/Arrhythmia Data Set/test_label.csv", header=None)
test_data_csv = pd.read_csv("Arrhythmia Data Set-20230503T055529Z-001/Arrhythmia Data Set/test_data.csv", header=None)


In [64]:
## 將資料和標籤分離
# X_train = train_data.iloc[:, :].values.astype(np.float32)
# y_train = train_label.iloc[:, -1].values
# X_test = test_data.iloc[:, :].values.astype(np.float32)
# y_test = test_label.iloc[:, -1].values


In [78]:
# 編碼標籤
le = LabelEncoder().fit(train_label_csv[0]) #將類別標籤進行編碼，將文字或類別數值轉換成模型可以理解的數值型別。
# y_train = le.fit_transform(y_train) #將訓練集的標籤進行編碼
y_test = test_label_csv[0].map(lambda s:-1 if s not in le.classes_ else s)
le.classes_=np.append(le.classes_, -1)
y_train=le.transform(train_label_csv[0])
y_test=le.transform(y_test)




In [67]:
#使用autoencoder進行降維處理
input_dim = train_data_csv.shape[1]
encoding_dim = 32

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="tanh", 
                activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder = Dense(int(encoding_dim / 2), activation="relu")(encoder)
encoded = Dense(int(encoding_dim / 4), activation="tanh")(encoder)
decoder = Dense(int(encoding_dim / 2), activation="tanh")(encoded)
decoder = Dense(input_dim, activation="relu")(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer="adam", loss="mean_squared_error")

autoencoder.fit(train_data_csv, train_data_csv,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_data=(test_data_csv, test_data_csv),
                verbose=1)


Epoch 1/50
3/3 [==============================] - 1s 199ms/step - loss: nan - val_loss: nan
Epoch 2/50
3/3 [==============================] - 0s 20ms/step - loss: nan - val_loss: nan
Epoch 3/50
3/3 [==============================] - 0s 20ms/step - loss: nan - val_loss: nan
Epoch 4/50
3/3 [==============================] - 0s 20ms/step - loss: nan - val_loss: nan
Epoch 5/50
3/3 [==============================] - 0s 20ms/step - loss: nan - val_loss: nan
Epoch 6/50
3/3 [==============================] - 0s 18ms/step - loss: nan - val_loss: nan
Epoch 7/50
3/3 [==============================] - 0s 21ms/step - loss: nan - val_loss: nan
Epoch 8/50
3/3 [==============================] - 0s 20ms/step - loss: nan - val_loss: nan
Epoch 9/50
3/3 [==============================] - 0s 19ms/step - loss: nan - val_loss: nan
Epoch 10/50
3/3 [==============================] - 0s 19ms/step - loss: nan - val_loss: nan
Epoch 11/50
3/3 [==============================] - 0s 19ms/step - loss: nan - val_loss: 

In [68]:
#使用autoencoder進行降維處理
encoder = Model(inputs=input_layer, outputs=encoded)
encoded_train = pd.DataFrame(encoder.predict(train_data_csv))
encoded_test = pd.DataFrame(encoder.predict(test_data_csv))


In [74]:
#訓練分類器
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=int(encoding_dim/4)))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(np.unique(y_train)), activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(encoded_train, y_train, epochs=50, batch_size=128)

# 預測標籤
y_pred = model.predict(encoded_test).argmax(axis=1)
num_classes = len(np.unique(y_train))


Epoch 1/50
3/3 [==============================] - 1s 4ms/step - loss: nan - accuracy: 0.6621
Epoch 2/50
3/3 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.6621
Epoch 3/50
3/3 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.6621
Epoch 4/50
3/3 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.6621
Epoch 5/50
3/3 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.6621
Epoch 6/50
3/3 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.6621
Epoch 7/50
3/3 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.6621
Epoch 8/50
3/3 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.6621
Epoch 9/50
3/3 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.6621
Epoch 10/50
3/3 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.6621
Epoch 11/50
3/3 [==============================] - 0s 3ms/step - loss

In [80]:
unknown_indexes = np.where(y_test == -1)[0]
unknown_x = test_data_csv.iloc[unknown_indexes]
print(y_test)


[0 0 5 0 1 1 1 0 0 0 1 0 4 1 7 0 0 0 0 2 0 0 5 0 0 4 0 0 0 0 0 0 0 0 0 0 2
 0 0 0 0 4 0 0 0 0 0 0 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 2 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0]


In [75]:
# 將未知類中的樣本進行分群並預測標籤
unknown_indexes = np.where(y_test == -1)[0]
unknown_x = test_data_csv.iloc[unknown_indexes]
unknown_x_encoded = encoder.predict(unknown_x)
kmeans = KMeans(n_clusters=len(np.unique(y_train)), random_state=0).fit(encoded_train)
unknown_labels = kmeans.predict(unknown_x_encoded)

# 將預測標籤與原始的分類器預測標籤合併
predicted_labels = np.hstack([y_pred, unknown_labels + num_classes])

# 將預測標籤轉換回原始的類別標籤
predicted_labels = le.inverse_transform(predicted_labels)

# 計算模型的準確率、召回率和F1分數
print(classification_report(test_label_csv[0], predicted_labels))

AttributeError: 'int' object has no attribute 'assign'

In [ ]:
# 將未知類中的樣本進行分群並預測標籤
unknown_x = x_test[unknown_indexes]
unknown_x_encoded = encoder.predict(unknown_x)
unknown_labels = kmeans.predict(unknown_x_encoded)

# 將預測標籤與原始的分類器預測標籤合併
predicted_labels = np.hstack([y_pred, unknown_labels + num_classes])


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# 計算混淆矩陣和分類報告
cm = confusion_matrix(y_test, predicted_labels)
cr = classification_report(y_test, predicted_labels)

print("Confusion Matrix:")
print(cm)
print("\nClassification Report:")
print(cr)
